In [1]:
!pip install crewai crewai_tools langchain_community ollama langchain

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew


In [4]:
import os
#from utils import get_openai_api_key, get_serper_api_key

#openai_api_key = get_openai_api_key()

open_api_key="NA"
os.environ["OPENAI_API_KEY"]=open_api_key
os.environ["OPENAI_MODEL_NAME"] = 'llama3:latest'

get_serper_api_key = "..."
os.environ["SERPER_API_KEY"] = get_serper_api_key

In [6]:
from langchain_ollama import OllamaLLM
from crewai import LLM

# Set Ollama base URL and model (Important: llama3 is not available in Ollama)
OLLAMA_BASE_URL = os.environ.get("OLLAMA_BASE_URL", "http://localhost:11434")

print("Base URL : " , OLLAMA_BASE_URL)

MODEL_NAME = "llama3:latest"  # Use a model AVAILABLE in your Ollama setup.

# Define the LLM (Correct Way)
llama3llm = OllamaLLM(base_url=OLLAMA_BASE_URL, model=MODEL_NAME)

# Connecting to a local LLM via Ollama
llama3llm = LLM(
    model="ollama/llama3",
    base_url="http://localhost:11434"
)

#print("llama3llm ", ollama_llm)

Base URL :  http://localhost:11434


In [7]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='sampleProfile.md')
semantic_search_resume = MDXSearchTool(mdx='sampleProfile.md')

Inserting batches in chromadb:   0%|                                                                                                        | 0/1 [00:00<?, ?it/s]


In [8]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    llm=llama3llm, 
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)


In [9]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llama3llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [10]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llama3llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [11]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llama3llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [12]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [13]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [14]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)


In [17]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/tayaln',
    'personal_writeup': """Nikhilesh is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [18]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Personal Profiler for Engineers# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.

## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/tayaln) URLs, and personal write-up (Nikhilesh is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innov

In [19]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**

**Summary:**
Highly skilled Software Engineer and Trainer with [X] years of experience in designing and delivering technical training programs. Proficient in mentoring diverse learners and teams to achieve technical excellence. Specializes in programming languages, software development methodologies, and modern tools. Passionate about empowering individuals and organizations through technology.

**Core Competencies:**

* Software Development Life Cycle (SDLC)
* Programming Languages (Python, Java, JavaScript, C++)
* Database Management (SQL, NoSQL)
* DevOps Practices & CI/CD Pipelines
* Agile and Scrum Methodologies
* Cloud Platforms (AWS, Azure, Google Cloud)
* Code Review and Debugging
* Curriculum Development and Instructional Design
* Team Mentoring and Knowledge Transfer

**Professional Experience:**

Software Engineer Trainer, [Company Name], [City, State]
[Month/Year] – Present

* Design and deliver comprehensive training programs in [specific programming languages/tools] for [specific audience, e.g., junior developers, corporate teams]
* Conduct live coding sessions, workshops, and hands-on labs to improve participants' practical coding abilities
* Mentor trainees in software engineering best practices, including clean code principles, version control systems, and unit testing
* Create and update training materials, including slides, coding exercises, and assessments, tailored to industry standards and emerging technologies
* Evaluate trainee progress through quizzes, projects, and performance reviews, offering personalized feedback

Key Achievements:

* Successfully trained over [number] developers, [percentage]% of whom achieved measurable career advancements
* Developed a flagship training module on [specific topic], adopted across the organization

Software Engineer, [Previous Company Name], [City, State]
[Month/Year] – [Month/Year]

* Designed and developed scalable software solutions, collaborating closely with cross-functional teams
* Conducted peer reviews and provided mentorship to junior developers, improving code quality and team efficiency
* Led internal workshops on [specific technology/tool], fostering team growth and knowledge sharing

Key Achievements:

* Reduced system downtime by [percentage]% by implementing robust monitoring solutions
* Spearheaded a migration to [specific platform/tool], improving team productivity

**Education:**

Bachelor’s Degree in Computer Science/Software Engineering, [University Name], [City, State] | [Year]

**Certifications:**

[Relevant Certification, e.g., AWS Certified Solutions Architect, Certified Scrum Master]
[Certification related to teaching/training, e.g., Train the Trainer (TTT)]

**Technical Skills:**

* Programming Languages: Python, Java, JavaScript, C++
* Frameworks: React, Angular, Django, Spring Boot
* Databases: MySQL, MongoDB, PostgreSQL
* Version Control: Git, GitHub, Bitbucket
* Tools: Jenkins, Docker, Kubernetes, Visual Studio Code

**Soft Skills:**

* Strong communication and presentation abilities
* Problem-solving and analytical mindset
* Patience and adaptability in training scenarios
* Leadership and mentorship

**Interests:**

* Open-source contribution
* Staying updated on emerging technologies
* Writing technical blogs/tutorials

In [20]:
display(Markdown("./interview_materials.md"))


This is my best complete final answer to the task.